<a href="https://colab.research.google.com/github/Aerospacerr/-PS-S3E13-/blob/main/StockAnalysis/Stock_Price_Scraper_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initialization and Imports

In [ ]:
# ### Initialization and Imports
# Update the system
!sudo apt -y update

# Install required packages
!sudo apt install -y wget curl unzip

# Download and install libu2f-udev dependency for Chrome
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb

# Download and install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb

# Download and install ChromeDriver
!wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver-linux64/chromedriver
!mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver

# Install selenium and chromedriver_autoinstaller
!pip install selenium chromedriver_autoinstaller
!pip install yfinance

from IPython.display import clear_output
#clear_output()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [861 kB]
Get

In [ ]:
from IPython.display import clear_output
import sys
import time
from urllib.parse import quote
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
import yfinance as yf
from google.colab import drive

In [ ]:
# Add chromedriver to system path
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


# Configure Chrome options for headless browsing
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# Install ChromeDriver automatically
chromedriver_autoinstaller.install()

# Initialize Chrome driver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
from urllib.parse import quote
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
import yfinance as yf

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
#Scraping
trade_data = []

In [ ]:
stopp

NameError: name 'stopp' is not defined

## Data

### Data Scraping

In [ ]:

driver.get('https://www.quiverquant.com/politicians/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

politician_links = []

for politician_link in soup.find_all('a', attrs={'class': 'flex-row-center'}):
    politician_links.append('https://www.quiverquant.com/' + quote(politician_link.get('href').replace('../', '')))

print(f'Found {len(politician_links)} politicians')
#print(f'First politician: {politician_links[0]}')

Found 0 politicians


In [ ]:
driver.get('https://www.quiverquant.com/congresstrading/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

politician_links = []

for politician_link in soup.find_all('a', attrs={'class': 'flex-row-center'}):
    politician_links.append('https://www.quiverquant.com/' + quote(politician_link.get('href').replace('../', '')))

print(f'Found {len(politician_links)} politicians')
print(f'First politician: {politician_links[0]}')

Found 88 politicians
First politician: https://www.quiverquant.com/congresstrading/politician/Josh%20Gottheimer-G000583


In [ ]:
pbar = tqdm(politician_links)

for politician_link in pbar:
    while True:
        driver.get(politician_link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        script_tag = soup.find('script', string=re.compile('let tradeData ='))

        try:
            script_content = script_tag.string
            pbar.set_description(f'Loaded {politician_link}')
            break
        except:
            pbar.set_description(f'Failed to load {politician_link}. Retrying in 10 seconds...')
            time.sleep(10)

    trade_data_match = re.search(r'let tradeData = (\[.*?\]);', script_content, re.DOTALL)
    trade_data_json = trade_data_match.group(1)

    trade_data_by_politician = json.loads(trade_data_json)

    for td in trade_data_by_politician:
        trade_data.append(
            {
                'Politician Profile': politician_link,
                'Stock Ticker': td[0],
                'Transaction': td[1],
                'Disclosed': td[2],
                'Traded': td[3],
                'Description': td[4],
                'Politician': td[6],
                'House Information': td[7],
                'Stock': td[8],
                'Amount Range': td[10],
                'Industry': td[13],
                'Amount': td[14]
            }
        )

    time.sleep(1)

### Data Exploration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the dataset
file_path_yahoo = "/content/drive/MyDrive/Upwork/Stock Data Analysis/all_stock_data.csv"
transactions_yahoo = pd.read_csv(file_path_yahoo)

# Display the first few rows of the dataframe
transactions_yahoo.sample(50)

In [ ]:
# Load the dataset
file_path = '/content/drive/MyDrive/Upwork/Stock Data Analysis/transaction_scraped.csv'
transactions = pd.read_csv(file_path)

# Display the first few rows of the dataframe
transactions.sample(50)

In [ ]:
transactions.isnull().sum()

In [ ]:
df = pd.DataFrame(transactions)

df = df.drop(columns=['Politician Profile', 'Disclosed', 'Description', 'House Information', 'Stock', 'Industry'])

df['Traded'] = pd.to_datetime(df['Traded'])

df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# Handle missing values before using str.contains
df[df['Stock Ticker'].notna() & df['Stock Ticker'].str.contains('\.')].nunique()

In [ ]:
# prompt: df.head() with stock ticker = (if any . in stock ticker)

df[df['Stock Ticker'].str.contains('\.')].head()


In [ ]:
import yfinance as yf

data = yf.download("AAPL", start="2020-01-01", end="2021-01-01")[['Close']]

# View the first few rows of the Close data
data.head()

In [ ]:
import yfinance as yf

tickers = ["AAPL", "MSFT", "GOOG"]  # Replace with your set of tickers
start_date = "2020-01-01"
end_date = "2021-01-01"

close_data = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)[['Close']]
    close_data[ticker] = data

# Access the Close data for a specific ticker
close_data

In [ ]:
recent_data = yf.download("BRK-B", period="5d")
recent_data

In [ ]:
#all_stock_data.to_csv('/content/drive/MyDrive/Upwork/Stock Data Analysis/all_stock_data.csv', index=False)

In [ ]:
unique_tickers

In [ ]:
data = yf.download('AAPL', start=start_date, end=end_date, auto_adjust=True)[['Close']]

In [ ]:
data

In [ ]:
yfinance_adj_close.head()

In [ ]:
# prompt: change stock ticker BRK.B to BRK-B in df

transactions['Stock Ticker'] = transactions['Stock Ticker'].replace('BRK.B', 'BRK-B')



# Clean and process tickers and dates
#transactions['Stock Ticker'] = transactions['Stock Ticker'].str.replace('$', '').str.strip()
transactions['Traded'] = pd.to_datetime(transactions['Traded']).dt.strftime('%Y-%m-%d')
transactions['Stock Ticker'] = transactions['Stock Ticker'].astype(str) # Convert 'Stock Ticker' column to string type


# Add columns for buy and sell prices
transactions['Transaction Price'] = transactions.apply(
    lambda row: get_stock_price(row['Stock Ticker'], row['Traded']), axis=1)

# Save the updated data with transaction prices
transactions.to_csv('transactions_with_prices.csv', index=False)
transactions.head()

In [ ]:
# Save the DataFrame to a CSV file in your Google Drive

df.to_csv('/content/drive/MyDrive/Upwork/Stock Data Analysis/transaction_scraped.csv', index=False)

In [ ]:
df.describe(include='all')

In [ ]:
# Summary statistics for numerical columns
df.describe()

In [ ]:
df.sample(10)

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
# Shape of the DataFrame
df.shape

In [ ]:
# Column names
df.columns

In [ ]:
# Data types of each column
df.dtypes

In [ ]:
# Number of non-null values in each column
df.count()

In [ ]:
df.describe()

In [ ]:
# Summary statistics for all columns, including categorical ones
df.describe(include='all')

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Unique values in the 'Transaction' column
df['Transaction'].unique()

In [ ]:
# Value counts for the 'Transaction' column
df['Transaction'].value_counts()

In [ ]:
# Filter the DataFrame for purchase transactions
purchase_df = df[df['Transaction'] == 'Purchase']

In [ ]:
# Group the data by 'Politician' and count the number of transactions
transactions_by_politician = df.groupby('Politician')['Transaction'].count()

In [ ]:
# Sort the transactions by politician in descending order
transactions_by_politician_sorted = transactions_by_politician.sort_values(ascending=False)

In [ ]:
# Group the data by 'Stock Ticker' and calculate the average amount
average_amount_by_stock = df.groupby('Stock Ticker')['Amount'].mean()

In [ ]:
# Sort the average amounts by stock in descending order
average_amount_by_stock_sorted = average_amount_by_stock.sort_values(ascending=False)

In [ ]:
# Filter the DataFrame for transactions related to a specific stock
specific_stock_df = df[df['Stock Ticker'] == 'AAPL']

In [ ]:
# Calculate the time difference between disclosure and trade dates
df['Disclosure_Trade_Difference'] = (df['Disclosed'] - df['Traded']).dt.days

In [ ]:
# Distribution of disclosure-trade time differences
df['Disclosure_Trade_Difference'].describe()

In [ ]:
df['Trade Date'] = df['Traded'].dt.date

## ROI Calculation

In [ ]:
# Prepare Data for ROI calculation
df_roi = df[df['Politician'] == 'Josh Gottheimer']

In [ ]:
df_roi

In [ ]:
# Group by 'Stock Ticker' and 'Transaction' and count the occurrences
transaction_counts = df_roi.groupby(['Stock Ticker', 'Transaction']).size().reset_index(name='Count')

# Display the result
print(transaction_counts)


# New section

In [ ]:
all_prices

In [ ]:
import pandas as pd



# Function to get the price of a stock on a specific date
def get_price_on_date(stock_ticker, date, prices_df):
    try:
        price_row = prices_df[(prices_df['Stock_Ticker'] == stock_ticker) & (prices_df['Traded'] == date)]
        if not price_row.empty:
            return price_row.iloc[0]['Price']
        else:
            return None
    except IndexError:
        return None

# Function to calculate the ROI
def calculate_roi(purchase_price, sell_price):
    return ((sell_price - purchase_price) / purchase_price) * 100

# Filter by a specific politician
politician_name = 'Politician1'
df_filtered = df_roi[df_roi['Politician'] == politician_name]

# Today's date and earliest available date
today_date = all_prices_df['Traded'].max()
earliest_date = all_prices_df['Traded'].min()

# Results storage
roi_results = []

# Group by Stock Ticker
grouped = df_filtered.groupby(['Stock_Ticker'])

for stock_ticker, group in grouped:
    purchases = group[group['Transaction'] == 'Purchase']
    sales = group[group['Transaction'] == 'Sale']

    if not purchases.empty and not sales.empty:
        # Match purchases and sales
        for (idx_p, purchase), (idx_s, sale) in zip(purchases.iterrows(), sales.iterrows()):
            buy_price = get_price_on_date(stock_ticker, pd.to_datetime(purchase['Traded']), all_prices_df)
            sell_price = get_price_on_date(stock_ticker, pd.to_datetime(sale['Traded']), all_prices_df)
            if buy_price is not None and sell_price is not None:
                roi = calculate_roi(buy_price, sell_price)
                roi_results.append({'Politician': politician_name, 'Stock_Ticker': stock_ticker, 'ROI': roi, 'Buy_Date': purchase['Traded'], 'Sell_Date': sale['Traded']})

    elif not purchases.empty:
        # Calculate ROI for buys with today's price
        today_price = get_price_on_date(stock_ticker, today_date, all_prices_df)
        for _, purchase in purchases.iterrows():
            buy_price = get_price_on_date(stock_ticker, pd.to_datetime(purchase['Traded']), all_prices_df)
            if buy_price is not None and today_price is not None:
                roi = calculate_roi(buy_price, today_price)
                roi_results.append({'Politician': politician_name, 'Stock_Ticker': stock_ticker, 'ROI': roi, 'Buy_Date': purchase['Traded'], 'Sell_Date': today_date})

    elif not sales.empty:
        # Calculate ROI for sales with earliest available price
        earliest_price = get_price_on_date(stock_ticker, earliest_date, all_prices_df)
        for _, sale in sales.iterrows():
            sell_price = get_price_on_date(stock_ticker, pd.to_datetime(sale['Traded']), all_prices_df)
            if earliest_price is not None and sell_price is not None:
                roi = calculate_roi(earliest_price, sell_price)
                roi_results.append({'Politician': politician_name, 'Stock_Ticker': stock_ticker, 'ROI': roi, 'Buy_Date': earliest_date, 'Sell_Date': sale['Traded']})

# Create a DataFrame with the ROI results
roi_df = pd.DataFrame(roi_results)

# Calculate summary statistics
total_roi = roi_df['ROI'].sum()
highest_roi = roi_df['ROI'].max()
lowest_roi = roi_df['ROI'].min()

print("Total ROI: {:.2f}%".format(total_roi))
print("Highest ROI: {:.2f}%".format(highest_roi))
print("Lowest ROI: {:.2f}%".format(lowest_roi))
print("\nResults DataFrame:")
print(roi_df)


In [ ]:
df_roi.nunique()

In [ ]:
yahoo_stock_roi_name = df_roi['Stock Ticker'].unique()

In [ ]:
yahoo_stock_roi_name

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf



# Function to calculate ROI
def calculate_roi(trades, historical_prices):
    roi_data = []
    for _, trade in trades.iterrows():
        symbol = trade['Stock Ticker']
        transaction_type = trade['Transaction']
        transaction_date = trade['Transaction Date']
        amount = float(trade['Amount'].replace('$', '').replace(',', ''))

        # Complete trade logic
        if transaction_type == 'Buy':
            buy_price = historical_prices[symbol].loc[transaction_date]['Close']
            sell_price = historical_prices[symbol].iloc[-1]['Close']  # Current price
            roi = (sell_price - buy_price) / buy_price * 100

        elif transaction_type == 'Sale':
            sell_price = historical_prices[symbol].loc[transaction_date]['Close']
            buy_price = historical_prices[symbol].iloc[0]['Close']  # Earliest price
            roi = (sell_price - buy_price) / buy_price * 100

        roi_data.append({
            'Politician': trade['Politician'],
            'Stock Ticker': symbol,
            'Transaction Type': transaction_type,
            'Transaction Date': transaction_date,
            'ROI': roi
        })

    return pd.DataFrame(roi_data)

# Main function to execute the workflow
def main():
    trades = df_roi
    unique_symbols = trades['Stock Ticker'].unique()
    historical_prices = {symbol: get_historical_prices(symbol) for symbol in unique_symbols}
    roi_data = calculate_roi(trades, historical_prices)
    roi_data.to_csv('congress_trading_roi.csv', index=False)

if __name__ == "__main__":
    main()


## YFinance Data Scrape & Merge


In [ ]:
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)[['Close']]

In [ ]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# Assuming 'transactions' is your DataFrame with a 'Stock Ticker' column


########
#######unique_tickers = transactions['Stock Ticker'].str.replace('$', '').str.strip().unique()

unique_tickers = yahoo_stock_roi_name[:10]
# Fetch historical prices for all unique tickers
start_date = "2021-01-01"
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')  # Today's date

# Initialize a dictionary to store the data
yfinance_adj_close = {}

# Use tqdm to display a progress bar
for ticker in tqdm(unique_tickers, desc="Fetching stock data"):
    try:
        # Download the adjusted close price
        data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)[['Close']]
        yfinance_adj_close[ticker] = data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")




In [ ]:
data

In [ ]:
aapl_data = yfinance_adj_close['AAPL']


In [ ]:
aapl_data

In [ ]:
yfinance_adj_close

In [ ]:
combined_data = pd.concat(yfinance_adj_close, axis=1)
combined_data.columns = combined_data.columns.droplevel(0)  # Dropping the top-level column index

# Save the combined data to a single CSV file
combined_data.to_csv('combined_stock_data.csv', index=True)

In [ ]:
# Initialize an empty DataFrame to store all yfinance data
all_data = pd.DataFrame()

# Normalize and clean the data if necessary
#transactions['Stock Ticker'] = transactions['Stock Ticker'].str.replace('$', '').str.strip()
transactions['Traded'] = pd.to_datetime(transactions['Traded'])

# Reset index to make 'Date' a column
all_data.reset_index(inplace=True)
yfinance_adj_close.rename(columns={'Date': 'Traded'}, inplace=True)

# Merge transactions with stock prices
merged_data = pd.merge(transactions, v, on=['Stock Ticker', 'Traded'], how='left')

# Save the merged data to a new CSV
merged_data.to_csv("merged_transactions_and_prices.csv", index=False)

merged_data.head()

In [ ]:
transactions.head()

In [ ]:
transactions.nunique()

In [ ]:
all_prices.sample(10)

In [ ]:
all_prices.Stock_Ticker

In [ ]:
all_prices.nunique()

In [ ]:
import pandas as pd

# Assuming yfinance_adj_close is a dictionary where keys are tickers and values are DataFrames
# containing 'Close' prices and indexed by date.

# List to hold individual DataFrames with a 'Traded' date column and a 'Stock_Ticker' column
dataframes = []

for ticker, df in yfinance_adj_close.items():
    # Reset the index to convert the Date index into a column
    df_reset = df.reset_index()
    df_reset.rename(columns={'index': 'Traded', 'Close': 'Price'}, inplace=True)
    df_reset['Stock_Ticker'] = ticker
    dataframes.append(df_reset)

# Concatenate all DataFrames into a single DataFrame
all_prices = pd.concat(dataframes, axis=0, ignore_index=True)

# Ensure 'Traded' column is treated as datetime
all_prices['Traded'] = pd.to_datetime(all_prices['Traded'], errors='coerce')

# Checking for any NaT values in 'Traded' after conversion
if all_prices['Traded'].isnull().any():
    print("Warning: Some 'Traded' dates could not be converted to datetime.")

# Print the first few rows to check the result
print(all_prices.head())


In [ ]:
import pandas as pd

# Example structure of yfinance_adj_close
# yfinance_adj_close = {
#     'MSFT': DataFrame, 'AAPL': DataFrame, ...
# }

# Convert dictionary to DataFrame
all_prices = pd.concat(
    [df.assign(Stock_Ticker=ticker) for ticker, df in yfinance_adj_close.items()],
    axis=0
).reset_index().rename(columns={'index': 'Traded', 'Close': 'Price'})

# Ensure 'Traded' is datetime
all_prices['Traded'] = pd.to_datetime(all_prices['Traded'])


In [ ]:
# Merge the transaction data with stock prices on 'Stock Ticker' and 'Traded'
merged_data = pd.merge(transactions, all_prices, left_on=['Stock Ticker', 'Traded'], right_on=['Stock_Ticker', 'Traded'], how='left')

# Save the merged data to a new CSV file
merged_data.to_csv("merged_transactions_and_prices.csv", index=False)

# Display the first few rows of the merged DataFrame
print(merged_data.head())


In [ ]:
yfinance_adj_close

In [ ]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# Assuming 'transactions' is your DataFrame with a 'Stock Ticker' column
unique_tickers = transactions['Stock Ticker'].str.replace('$', '').str.strip().unique()[:10]

# Fetch historical prices for all unique tickers
start_date = "2021-01-01"
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')  # Today's date

# Initialize a dictionary to store the data
yfinance_adj_close = {}

# Use tqdm to display a progress bar
for ticker in tqdm(unique_tickers, desc="Fetching stock data"):
    try:
        # Download the adjusted close price
        data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)[['Close']]
        yfinance_adj_close[ticker] = data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Example to check the data for a specific ticker
for ticker, data in yfinance_adj_close.items():
    print(f"Data for {ticker}:")
    print(data.head())


In [ ]:
yfinance_adj_close.items()

In [ ]:
import pandas as pd

# Sample dictionary structure for yfinance_adj_close
# yfinance_adj_close = {
#     'AAPL': DataFrame with date index and 'Close' price column,
#     'MSFT': DataFrame with date index and 'Close' price column,
#     # ... other tickers
# }

# List to store individual DataFrames
dataframes = []

# Iterate over each ticker and its corresponding DataFrame
for ticker, df in yfinance_adj_close.items():
    # Reset index to turn the date index into a column
    df_reset = df.reset_index()
    df_reset.rename(columns={'Date': 'Traded', 'Close': 'Price'}, inplace=True)
    df_reset['Stock_Ticker'] = ticker
    dataframes.append(df_reset)

# Concatenate all DataFrames into a single DataFrame
all_prices = pd.concat(dataframes, axis=0, ignore_index=True)

# Convert 'Traded' to datetime format
all_prices['Traded'] = pd.to_datetime(all_prices['Traded'], errors='coerce')

# Display the first few rows to verify the result
print(all_prices.head())


In [ ]:
all_prices

In [ ]:
# Reshape the DataFrame with 'Traded' as index and 'Stock_Ticker' as columns
pivot_df = all_prices.pivot(index='Traded', columns='Stock_Ticker', values='Price')

# Display the first few rows of the pivoted DataFrame
print(pivot_df.head())


In [ ]:
pivot_df

### Merge df_roi and pivot_stock_prices

In [ ]:
df_roi

In [ ]:
df_roi

In [ ]:
pivot_df.head()

In [ ]:
pivot_df.reset_index()

In [ ]:
pivot_df

In [ ]:
pivot_df.head()

In [ ]:
pivot_df_reset

In [ ]:
df_roi.info()

In [ ]:
pivot_df.head()

In [ ]:
all_prices.info()

In [ ]:
import pandas as pd

# Ensure 'Stock Ticker' is a string in df_roi
df_roi['Stock Ticker'] = df_roi['Stock Ticker'].astype(str)

"""# Reset index in pivot_df to include 'Traded' (Date) as a column
pivot_df_reset = pivot_df.reset_index()
pivot_df_reset['Date'] = pivot_df_reset['Traded']
#df_roi = df_roi.reset_index(drop=False)
pivot_df_reset['Date'] = pd.to_datetime(pivot_df_reset['Date'], format='%Y-%m-%d')"""

# Merge df_roi with pivot_df_reset on 'Traded' and 'Stock Ticker'
merged_df = pd.merge(df_roi, all_prices, left_on=['Traded', 'Stock Ticker'], right_on=['Traded', 'Stock_Ticker'], how='left')

# Drop the redundant 'Date' column if needed
#merged_df.drop(columns=['Date'], inplace=True)

# Display the merged DataFrame
merged_df.head()


In [ ]:
# Delete the specified columns
merged_df = merged_df.drop(columns=['Politician Profile', 'Description', 'House Information', 'Stock', 'Industry'])
#merged_df = merged_df.dropna()

In [ ]:
deneme_df = merged_df[merged_df['Stock_Ticker'] == "AAPL"]

In [ ]:
deneme_df.head()

In [ ]:
import pandas as pd
from datetime import datetime



# Define a function to calculate ROI
def calculate_roi(buy_price, sell_price):
    return ((sell_price - buy_price) / buy_price) * 100

# Define a function to calculate the highest ROI
def highest_roi_for_stock(stock_df):
    buys = stock_df[stock_df['Transaction'] == 'Purchase']
    sells = stock_df[stock_df['Transaction'] == 'Sale']

    if not buys.empty and not sells.empty:
        # Both buy and sell trades present
        max_roi = float('-inf')
        for _, buy_row in buys.iterrows():
            for _, sell_row in sells.iterrows():
                roi = calculate_roi(buy_row['Price'], sell_row['Price'])
                if roi > max_roi:
                    max_roi = roi
        return max_roi

    elif not buys.empty:
        # Only buy trades are present
        latest_price = buys['Price'].max()
        today_price = df[df['Traded'] == df['Traded'].max()]['Price'].values[0]
        roi = calculate_roi(latest_price, today_price)
        return roi

    elif not sells.empty:
        # Only sell trades are present
        earliest_price = df[df['Traded'] == df['Traded'].min()]['Price'].values[0]
        max_sell_price = sells['Price'].max()
        roi = calculate_roi(earliest_price, max_sell_price)
        return roi

    else:
        return None

# Calculate highest ROI for each stock ticker
results = []
for ticker in df['Stock Ticker'].unique():
    stock_df = df[df['Stock Ticker'] == ticker]
    roi = highest_roi_for_stock(stock_df)
    results.append({'Stock Ticker': ticker, 'Highest ROI': roi})

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
all_prices

In [ ]:
import pandas as pd

def get_price_on_date(stock_ticker, date, all_prices_df):
    # Find the price on the given date for the specified stock ticker
    price_df = all_prices_df[(all_prices_df['Stock_Ticker'] == stock_ticker) & (all_prices_df['Traded'] == date)]
    if not price_df.empty:
        return price_df['Price'].values[0]
    return None

def calculate_roi(purchase_amount, purchase_price, sell_amount, sell_price):
    # Calculate ROI as a percentage
    return ((sell_price - purchase_price) / purchase_price) * 100

def highest_roi(politician_df, all_prices_df):
    results = []

    # Group by Stock_Ticker and Politician
    grouped = politician_df.groupby(['Stock_Ticker', 'Politician'])

    for (stock_ticker, politician), trades in grouped:
        buy_trades = trades[trades['Transaction'] == 'Purchase']
        sell_trades = trades[trades['Transaction'] == 'Sale']

        buy_dates = buy_trades['Traded'].tolist()
        sell_dates = sell_trades['Traded'].tolist()

        if buy_trades.empty and not sell_trades.empty:
            # Scenario 3: Only sell trades - Assume buying at the earliest date
            earliest_date = all_prices_df['Traded'].min()
            earliest_price = get_price_on_date(stock_ticker, earliest_date, all_prices_df)
            if earliest_price is not None:
                for _, sell_trade in sell_trades.iterrows():
                    sell_price = get_price_on_date(stock_ticker, sell_trade['Traded'], all_prices_df)
                    if sell_price is not None:
                        roi = calculate_roi(sell_trade['Amount'], earliest_price, sell_trade['Amount'], sell_price)
                        results.append({'Politician': politician, 'Stock_Ticker': stock_ticker, 'ROI': roi})

        elif not buy_trades.empty and sell_trades.empty:
            # Scenario 2: Only buy trades - Assume selling at today's price
            today_date = all_prices_df['Traded'].max()
            today_price = get_price_on_date(stock_ticker, today_date, all_prices_df)
            if today_price is not None:
                for _, buy_trade in buy_trades.iterrows():
                    buy_price = get_price_on_date(stock_ticker, buy_trade['Traded'], all_prices_df)
                    if buy_price is not None:
                        roi = calculate_roi(buy_trade['Amount'], buy_price, buy_trade['Amount'], today_price)
                        results.append({'Politician': politician, 'Stock_Ticker': stock_ticker, 'ROI': roi})

        elif not buy_trades.empty and not sell_trades.empty:
            # Scenario 1: Both buy and sell trades - Match the buys and sells
            for _, buy_trade in buy_trades.iterrows():
                buy_price = get_price_on_date(stock_ticker, buy_trade['Traded'], all_prices_df)
                if buy_price is not None:
                    # Match with a corresponding sell trade
                    for _, sell_trade in sell_trades.iterrows():
                        sell_price = get_price_on_date(stock_ticker, sell_trade['Traded'], all_prices_df)
                        if sell_price is not None:
                            roi = calculate_roi(buy_trade['Amount'], buy_price, sell_trade['Amount'], sell_price)
                            results.append({'Politician': politician, 'Stock_Ticker': stock_ticker, 'ROI': roi})

    results_df = pd.DataFrame(results)

    return results_df


In [ ]:
# Assuming deneme_df and all_prices_df are already defined and populated
highest_roi(deneme_df, all_prices)

In [ ]:
import pandas as pd

# Assuming you have already defined and computed results_df
results_df = highest_roi(deneme_df, all_prices)

# Calculate summary statistics
total_roi = results_df['ROI'].sum()
highest_roi = results_df['ROI'].max()
lowest_roi = results_df['ROI'].min()

# Print the results
print("Total ROI: {:.2f}%".format(total_roi))
print("Highest ROI: {:.2f}%".format(highest_roi))
print("Lowest ROI: {:.2f}%".format(lowest_roi))
print("\nResults DataFrame:")
print(results_df)

In [ ]:
len(deneme_df)

In [ ]:
deneme_df.head(50)

In [ ]:
results_df